## Web scraping
To get values from websites which don't provide an API is often only through scraping. It can be very tricky to get to the right values but this example here should help you to get started. This is very similar to the work-flow the [`scrape` sensor](https://home-assistant.io/components/sensor.scrape/) using.

### Get the value

Importing the needed modules.

In [144]:
import requests
from bs4 import BeautifulSoup

We want to scrape the counter for all our implementations from the [Component overview](https://home-assistant.io/components/).

The section (extracted from the source) which is relevant for this example is shown below.

```html
...
<div class="grid__item one-sixth lap-one-whole palm-one-whole">
<div class="filter-button-group">
<a href='#all' class="btn">All (444)</a>
<a href='#featured' class="btn featured">Featured</a>
<a href='#alarm' class="btn">
Alarm
(9)
</a>
...
```

The line `<a href='#all' class="btn">All (444)</a>` contains the counter. 

In [145]:
URL = 'https://home-assistant.io/components/'

With `requests` the website is retrieved and with `BeautifulSoup` parsed.

In [146]:
raw_html = requests.get(URL).text
raw_data = BeautifulSoup(raw_html, 'html.parser')

Now you have the complete content of the page. [CSS selectors](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors) can be used to identify the counter. We have several options to get the part in question. As `BeautifulSoup` is giving us a list with the findings, we only need to identify the position in the list.

In [162]:
print(raw_data.select('a')[7])

<a class="btn" href="#all">All (444)</a>


In [161]:
print(raw_data.select('a:nth-of-type(8)'))

[<a class="btn" href="#all">All (444)</a>]


In [140]:
print(raw_data.select('a[href="#all"]'))

[<a class="btn" href="#all">All (444)</a>]


In [141]:
print(raw_data.select('.btn')[0])

<a class="btn" href="#all">All (444)</a>


We only need the actual text.

In [142]:
print(raw_data.select('.btn')[0].text)

All (444)


This is a string and can be manipulated. We focus on the number.

In [143]:
print(raw_data.select('.btn')[0].text[5:8])

444


This is the number of the current platforms/component from the [Component overview](https://home-assistant.io/components/) which are available in Home Assistant.

The values you identified here can be re-used to configure a [`scrape` sensor](https://home-assistant.io/components/sensor.scrape/).

### Send the value to the Home Assistant frontend
The ["Using the Home Assistant Python API"](http://nbviewer.jupyter.org/github/home-assistant/home-assistant-notebooks/blob/master/home-assistant-python-api.ipynb) notebooks contains an intro to the [Python API](https://home-assistant.io/developers/python_api/) of Home Assistant and Jupyther notebooks. Here we are sending the scrapped value to the Home Assistant frontend.

In [134]:
import homeassistant.remote as remote

HOST = '127.0.0.1'
PASSWORD = 'YOUR_PASSWORD'

api = remote.API(HOST, PASSWORD)

In [135]:
new_state = raw_data.select('.btn')[0].text[5:8]
attributes = {
  "friendly_name": "Gitter Users",
  "unit_of_measurement": "Count"
}
remote.set_state(api, 'sensor.gitter_users', new_state=new_state, attributes=attributes)

True